In [ ]:
import string
import os
import datetime
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from scipy.linalg import svd
from sklearn import ensemble
import scipy.linalg as la
import scipy.sparse.linalg as sla
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from time import time
from sklearn import metrics

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
start_date = pd.to_datetime('1957-03-01')
end_date =  start_date + pd.offsets.DateOffset(years=18)
ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
ytrain = Y_all[ind_training]
xtrain = X_all[ind_training]

In [ ]:
start_date =  end_date
end_date = start_date + pd.offsets.DateOffset(years=12)
ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
ytest = Y_all[ind_vali]
xtest= X_all[ind_vali]

In [ ]:
start_date =  end_date
end_date = start_date + pd.offsets.DateOffset(years=30)
ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
#print(type(ind_test))
yoos = Y_all[ind_test]
xoos= X_all[ind_test]

In [ ]:
#######Monthly Demean %%%
ytrain_demean= ytrain - np.mean(ytrain);
ytest_demean=ytest-np.mean(ytest);
mtrain=np.mean(ytrain);
mtest=np.mean(ytest);

sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
for i in range(len(xtrain.columns)):
    s=np.std(xtrain.iloc[:,[i]])
    colnames = xtrain.columns
    if s.values > 0:
        colname = colnames[i]
        xtrain.loc[:,colname]=xtrain[colname]/s.values
        xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
        xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
        sd[i]=s.values
    #### Start to Train %%%
    #model fitting

In [ ]:
lamv=np.arange(-1,0,0.2)
    #lamv = -0.6

r=np.zeros((len(lamv),3))

for n1 in range(len(lamv)):
    lr=10**lamv[n1]
    alpha=2
    ne=50
    clf = ensemble.GradientBoostingRegressor(loss = 'ls',learning_rate =lr,n_estimators = ne,criterion = 'friedman_mse',max_depth = 2)
    clffit = clf.fit(xtrain,np.ravel(ytrain))
    yhatbig1 = clffit.predict(xtest)
    r[n1,0]=1 - np.linalg.norm(yhatbig1-ytest['RET'])**2 / np.linalg.norm((ytest['RET']-mtrain.values))**2
    yhatbig1= clffit.predict(xoos)
    r[n1,1]=1-np.linalg.norm((yhatbig1-yoos['RET']))**2/np.linalg.norm((yoos['RET']-mtrain.values))**2
    yhatbig1= clffit.predict(xtrain)
    r[n1,2]=1-np.linalg.norm((yhatbig1-ytrain['RET']))**2/np.linalg.norm((ytrain['RET']-mtrain.values))**2

r2_oos=r[np.argmax(r[:,0]),1]
r2_is=r[np.argmax(r[:,0]),2]
print('GBRT R2 : '+np.str0(r2_oos) )

In [ ]:
for i in range(len(X_all.columns)):
    xtrain_1=xtrain.drop(columns=[X_all.columns[i]])
    xtest_1=xtest.drop(columns=[X_all.columns[i]])
    xoos_1= xoos.drop(columns=[X_all.columns[i]])
    
    lamv=np.arange(-1,0,0.2)
    #lamv = -0.6

    r=np.zeros((len(lamv),3))

    for n1 in range(len(lamv)):
        lr=10**lamv[n1]
        alpha=2
        ne=50
        clf = ensemble.GradientBoostingRegressor(loss = 'ls',learning_rate =lr,n_estimators = ne,criterion = 'friedman_mse',max_depth = 2)
        clffit = clf.fit(xtrain_1,np.ravel(ytrain))
        yhatbig1 = clffit.predict(xtest_1)
        r[n1,0]=1 - np.linalg.norm(yhatbig1-ytest['RET'])**2 / np.linalg.norm((ytest['RET']-mtrain.values))**2
        yhatbig1= clffit.predict(xoos_1)
        r[n1,1]=1-np.linalg.norm((yhatbig1-yoos['RET']))**2/np.linalg.norm((yoos['RET']-mtrain.values))**2
        yhatbig1= clffit.predict(xtrain_1)
        r[n1,2]=1-np.linalg.norm((yhatbig1-ytrain['RET']))**2/np.linalg.norm((ytrain['RET']-mtrain.values))**2

    r2_oos_1=r[np.argmax(r[:,0]),1]
    r2_is_1=r[np.argmax(r[:,0]),2]
    d=r2_oos-r2_oos_1
    print(X_all.columns[i]+':'+np.str0(d))

<!--
beta:0.08161700490291413
betasq:-0.19268278289813434
chmom:-0.020923252826167804
dolvol:-0.12781794858805373
idiovol:-0.22363859661838514
indmom:-0.03625675816535656
mom1m:0.03556924168681741
mom6m:-0.056875415400727825
mom12m:-0.16815955022771956
mom36m:0.07324836324374706
pricedelay:-0.00023237962716127214
turn:-0.04542008082879789
absacc:-0.12718875857158762
acc:-0.07774959550190541
age:-0.14947300202164948
agr:0.10561211167343387
cashdebt:-0.0414941385970522
cashpr:0.05833135293142311
cfp:-0.023556726017946783
cfp_ia:-0.0942818125004683
chatoia:-0.11846718306835635
chcsho:-0.07081942904391658
chempia:-0.28261229377499664
chinv:-0.20345544456457265
chpmia:-0.19814693482513235
convind:0.004702192804389327
currat:-0.1263059017308783
depr:-0.1463340997062601
divi:-0.02372441580849305
divo:-0.05561388690798408
dy:-0.07304196867430868
egr:-0.027545577089882034
ep:-0.19357724242494978
gma:-0.22766746536537896
grcapx:0.14278501950492295
grltnoa:-0.05398517803893066
herf:-0.12042199368842788
hire:-0.023717201583850933
invest:-0.13791957500086482
lev:-0.14157085935840863
lgr:-0.30320679956250185
mve_ia:-0.19322601124103977
operprof:-0.135416822337441
orgcap:-0.2370250387465811
pchcapx_ia:-0.041121465851054584
pchcurrat:-0.02910783677481099
pchdepr:-0.007571524773971827
pchgm_pchsale:-0.1794899874400726
pchquick:-0.15985486096200052
pchsale_pchinvt:-0.15707858189646284
pchsale_pchrect:-0.1885917089263156
pchsale_pchxsga:-0.060549314690300005
pchsaleinv:-0.23841771222533126
pctacc:-0.22753963993315107
ps:0.17420869189338983
quick:-0.22396476295815937
rd:-0.26567147814547587
rd_mve:0.03156951437191946
rd_sale:-0.15289279611047402
realestate:-0.10451560645979718
roic:-0.14493583388228792
salecash:0.13142690238182952
saleinv:-0.12958173717870336
salerec:-0.14559483075849178
secured:-0.22456905843292363
securedind:-0.12143871805930306
sgr:-0.032531666763630174
sin:-0.05180608979542045
sp:-0.1196901309874483
tang:-0.11940907423509461
tb:0.011844295164260554
aeavol:-0.10316739095209071
cash:-0.07101769502519639
chtx:-0.07161122645779505
cinvest:0.06561325592531819
ear:-0.0034493010131289026
nincr:-0.22201963078539366
roaq:-0.15121483344200204
roavol:-0.11064924696753753
roeq:-0.35138049666605164
rsup:-0.15767044356740811
stdacc:-0.09712427642220156
stdcf:-0.12095717422169727
ms:0.0007879174352170004
baspread:-0.1644322563681695
ill:0.14059608276133018
maxret:-0.221929687920694
retvol:-0.030525044818140223
std_dolvol:-0.04067477565258759
std_turn:-0.20263326259204817
zerotrade:-0.12423475576629928
bm:-0.06006498046053865
bm_ia:-0.20364758844626696
sic2_10:-0.05415765653852356
sic2_13:-0.1293714609170311
sic2_15:-0.015989226050264538
sic2_17:-0.2030986775769983
sic2_20:-0.12850983872877952
sic2_22:-0.014829030683344069
sic2_23:-0.09768237166161375
sic2_24:-0.12673288221314438
sic2_25:-0.22564746831190474
sic2_26:-0.058479373798578926
sic2_27:-0.11721453932164883
sic2_28:-0.15922068291750158
sic2_29:-0.22740683137037498
sic2_30:-0.13368833660774238
sic2_32:-0.08705304754768317
sic2_33:-0.10327292577891489
sic2_34:-0.024624200083044112
sic2_35:-0.05960715073177081
sic2_36:-0.23866156186536003
sic2_37:-0.13002327255597645
sic2_38:-0.023109167347349135
sic2_39:-0.05987123202320932
sic2_40:-0.11408672346073923
sic2_41:-0.22839734719251004
sic2_42:-0.13503583560935173
sic2_44:-0.01292650451952504
sic2_45:-0.14418877814478437
sic2_47:-0.22735120602070102
sic2_48:-0.024539251728568523
sic2_49:0.08996350433278621
sic2_50:-0.12004672142063999
sic2_51:0.08522370525493539
sic2_52:-0.16178175476440293
sic2_53:0.07633418350004506
sic2_54:-0.020585846543067277
sic2_56:-0.035990880089967536
sic2_57:-0.017065085237424782
sic2_58:-0.1345626014470156
sic2_59:-0.12116320180962914
sic2_60:-0.05293653350904837
sic2_61:-0.13429148303017202
sic2_62:-0.06070723758026553
sic2_63:-0.03340023199721065
sic2_64:-0.11498582287188786
sic2_65:-0.12601581524718908
sic2_67:0.16790956516501998
sic2_70:-0.10277825327818091
sic2_72:0.083650138336119
sic2_73:-0.2278274468910655
sic2_75:-0.20210396743555803
sic2_78:-0.02123175076104955
sic2_79:-0.1997230291940837
sic2_80:-0.12828062702496323
sic2_87:-0.22689562601043933
sic2_99:-0.15518220055535403